# Visuzalize Tensorpack Mask-RCNN Detection Results

This notebook visualizes detection results predicted by a trained [Tensorpack Mask-RCNN](https://github.com/tensorpack/tensorpack/tree/master/examples/FasterRCNN) model. 
## Load trained model

First we define the system path for Python classes.

In [ ]:
import os, glob
import sys

sys.path.append('/tensorpack/examples/FasterRCNN')

Next, we initialize the ResNet FPN Mask RCNN model.

In [ ]:
from modeling.generalized_rcnn import ResNetFPNModel

# create a mask r-cnn model
mask_rcnn_model = ResNetFPNModel()

Next we list the log directories to select the directory with a model checkpoint we want to load. The general patterm for the `model_dir` below is `/efs/maskrcnn-yyyy-mm-dd-hh-mm-ss/train_log/maskrcnn/`

In [ ]:
!ls -ltr /efs/maskrcnn-*

Next we specify the `model_dir` below and load the `trained model`. 

In [ ]:
import glob

# find best pre-trained model checkpoint
latest_trained_model = ""
model_dir= # e.g. "/efs/maskrcnn-2020-02-24-19-48-17/train_log/maskrcnn/"
model_search_path = os.path.join(model_dir, "model-*.index" )
for model_file in glob.glob(model_search_path):
    if model_file > latest_trained_model:
        latest_trained_model = model_file

trained_model = latest_trained_model[:-6]
print(f'Using model: {trained_model}')

Next we initialize the model configuration to match the configuration we used for training.

In [ ]:
from config import finalize_configs, config as cfg
from dataset import register_coco

# setup config
cfg.MODE_FPN = True
cfg.MODE_MASK = True
cfg.TEST.RESULT_SCORE_THRESH = cfg.TEST.RESULT_SCORE_THRESH_VIS
cfg.DATA.BASEDIR = '/efs/data/'
register_coco(cfg.DATA.BASEDIR)
finalize_configs(is_training=False)

## Create Predictor

Next we create a predictor that uses our trained model to make predictions on test images.

In [ ]:
from tensorpack.predict.base import OfflinePredictor
from tensorpack.tfutils.sessinit import get_model_loader
from tensorpack.predict.config import PredictConfig

# Create an inference predictor
# PredictConfig takes a model, input tensors and output tensors
input_tensors = mask_rcnn_model.get_inference_tensor_names()[0]
output_tensors = mask_rcnn_model.get_inference_tensor_names()[1]
            
predictor = OfflinePredictor(PredictConfig(
                model=mask_rcnn_model,
                session_init=get_model_loader(trained_model),
                input_names=input_tensors,
                output_names=output_tensors))

## Download COCO Test 2017 dataset
Below we download the [COCO 2017 Test dataset](http://cocodataset.org/#download) and extract the downloaded dataset.

In [ ]:
!wget -O /tmp/test2017.zip http://images.cocodataset.org/zips/test2017.zip

Next, we extract the downloaded COCO 2017 Test dataset. This step executes quietly and may take couple of minutes.

In [ ]:
!unzip -q -d /tmp/ /tmp/test2017.zip
!rm  /tmp/test2017.zip

## Load test image

Next, we find a random image to test from COCO 2017 Test dataset. You can come back to this cell when you want to load the next test image.

In [ ]:
import os
import random

test2017_dir=os.path.join("/tmp", "test2017")
img_id=random.choice(os.listdir(test2017_dir))
img_local_path = os.path.join(test2017_dir,img_id)
print(img_local_path)

Next, we load the random test image and convert the image color scheme from BGR to RBG.

In [ ]:
import cv2

img=cv2.imread(img_local_path, cv2.IMREAD_COLOR)
print(img.shape)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

Next. we show the raw image we randomly loaded from the COCO 2017 Test dataset.

In [ ]:
import matplotlib.pyplot as plt

fig,ax = plt.subplots(figsize=(img.shape[1]//50, img.shape[0]//50))
ax.imshow(img.astype(int))
plt.show()

## Predict
Next, we use the predictor to predict detection results.

In [ ]:
from eval import predict_image, DetectionResult
detection_results = predict_image(img, predictor)

Next we draw the detection results on our image.

In [ ]:
from viz import draw_final_outputs
final_viz = draw_final_outputs(img, detection_results)

Next we visualize the detection results drawn on the image.

In [ ]:
fig,ax = plt.subplots(figsize=(final_viz.shape[1]//50, final_viz.shape[0]//50))
ax.imshow(final_viz.astype(int))

Go back to <b>Load test image</b> cell if you want to test more images.